In [ ]:
import torch
import torch.nn.functional as F

embeddings = torch.load("../outputs/embeddings/OOPS_cs_16@7_5.pt")

In [ ]:
embeddings.shape

In [ ]:
embeddings[0]

In [ ]:
F.cosine_similarity(embeddings[0], embeddings[1], dim=0)

In [ ]:
(embeddings @ embeddings.T).shape